In [ ]:
!pip install --upgrade tensorflow-federated
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    sys.path.append("./drive/MyDrive/Colab Notebooks/Projektarbeit_2/")
    BASE_DIR = sys.path[-1]
    !pip install --upgrade tensorflow-federated
    !pip install nest_asyncio
    import nest_asyncio
    nest_asyncio.apply()
else: 
    BASE_DIR = "../"
    sys.path.append(BASE_DIR)

import json
import numpy as np 
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.model_selection import train_test_split
import collections
from tensorflow.keras import Model, callbacks
from tensorflow.keras.layers import Dense, Softmax
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import random
import time
from Reader import Reader
from model.FLModel import FLModel
from model.BNModel import BNModel
from Utils import Utils
import statistics
from datetime import datetime

%load_ext tensorboard

In [ ]:
def read_config():
    config_file = BASE_DIR + "config/config.json"
    config = None
    with open(config_file) as json_file:
        config = json.loads(json_file.read())
    return config

def split_input_target(input, target):
    return input, target

def create_dataset(x, y, use_tff = True):
    ds =  tf.data.Dataset.from_tensor_slices((x, y))
    
    if use_tff:
        return (
        ds.repeat(EPOCHS).shuffle(SHUFFLE_BUFFER)
        .map(split_input_target)).batch(BATCH_SIZE) 
    else:
        return ds.repeat(BATCH_SIZE * EPOCHS).shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE,drop_remainder = True) 

def get_split(x, y):
    return train_test_split(x, y, test_size=0.2, random_state=42)

def create_unfederated_dataset(x, features):
    former_shape = x[:, start_id:features].shape
    client_x = np.delete( x[:, start_id:features], label_id-1, 1 ).reshape(former_shape[0], former_shape[1]-1)
    client_x = scaler.transform(client_x)
    client_y = x[:, label_id].reshape(-1, 1)
    X_train, X_test, y_train, y_test = get_split(client_x, client_y)
    X_train, X_val, y_train, y_val = get_split(X_train, y_train)
    train_data = create_dataset(X_train, y_train, use_tff=False)
    test_data = create_dataset(X_test, y_test, use_tff=False)
    val_data = create_dataset(X_val, y_val, use_tff=False)
    return train_data, test_data, val_data

In [ ]:
def format_size(size):
  """A helper function for creating a human-readable size."""
  size = float(size)
  for unit in ['bit','Kibit','Mibit','Gibit']:
    if size < 1024.0:
      return "{size:3.2f}{unit}".format(size=size, unit=unit)
    size /= 1024.0
  return "{size:.2f}{unit}".format(size=size, unit='TiB')

def set_sizing_environment():
  """Creates an environment that contains sizing information."""
  # Creates a sizing executor factory to output communication cost
  # after the training finishes. Note that sizing executor only provides an
  # estimate (not exact) of communication cost, and doesn't capture cases like
  # compression of over-the-wire representations. However, it's perfect for
  # demonstrating the effect of compression in this tutorial.
  sizing_factory = tff.framework.sizing_executor_factory()

  # TFF has a modular runtime you can configure yourself for various
  # environments and purposes, and this example just shows how to configure one
  # part of it to report the size of things.
  context = tff.framework.ExecutionContext(executor_fn=sizing_factory)
  tff.framework.set_default_context(context)

  return sizing_factory

In [ ]:
config = read_config()
BATCH_SIZE = config["BATCH_SIZE"]
PREFETCH_BUFFER = config["PREFETCH_BUFFER"]
SHUFFLE_BUFFER = config["SHUFFLE_BUFFER"]
CLIENTS = config["CLIENTS"]
DATA_DIR = config["DATA_DIR"]
OUT_DIR = config["OUT_DIR"]
LOG_DIR = config["LOG_DIR"]
EPOCHS = config["EPOCHS"] 
NUM_CLASSES = config["num_classes_app_usages"]
file = config["file_app_usages"]
drop_index = True
if "infected" in file:
    drop_index = False

if IN_COLAB:
    tf_log_dir = "../tmp/logs/scalars/" + datetime.now().strftime("%Y%m%d") + "/"
    !rm -R /tmp/logs/scalars/*
else:
    tf_log_dir = LOG_DIR + "tensorboard/scalars/" + datetime.now().strftime("%Y%m%d") + "/"
    
if ("infected" in file):
    start_id = 1
    label_id = 9
elif ("Cosphere" in file):
    start_id = 1
    label_id = 4
else:
    start_id = 1
    label_id = 9

In [ ]:
k = 10
use_bn = True
use_tff = True
learning_rate  = 1e-1
momentum = 0.9
nesterov = False

entropy_loss = tf.keras.losses.SparseCategoricalCrossentropy()
sparseCategoricalAcc = tf.keras.metrics.SparseCategoricalAccuracy()
sparseTopKCategoricalAccuracy = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=k)
client_optimizer =  tf.keras.optimizers.SGD(learning_rate= learning_rate, momentum=momentum, nesterov=nesterov)
server_optimzer = tf.keras.optimizers.SGD(learning_rate= 1.0)

In [ ]:
client_ids = None
scaler = StandardScaler()
utils = Utils()
reader = Reader(BASE_DIR + DATA_DIR, file, drop_index)
data = reader.get_data()

if ("IID" in file): 
    data = utils.create_clients(data, CLIENTSs, strict = False)
    reader.set_features(reader.get_features() + 1)
    client_ids =  [i for i in range(0, CLIENTS)]
    
cols = [i for i in range(0, reader.get_features())]
del cols[0]
if "app" in file:
    del cols[2]
elif "infected" in file:
    del cols[8]
elif "Cosphere" in file:
    del cols[3]

features = reader.get_features()
scaler.fit(data[:, cols])

if ((file == "App_usage_trace.txt") or (file == "top_90_apps.csv") or ("infected" in file) or ("Cosphere" in file)): 
    data  = utils.map_ids(data.copy())
    num_of_users = int((np.amax(data[:, 0]) + 1))
    client_ids = list(range(0, num_of_users))
    random.shuffle(client_ids)
    client_ids = client_ids[:CLIENTS]
client_ids = sorted(client_ids)

In [ ]:
train_data = []
test_data = []
    
for id in client_ids:
    indicees = data[:, 0] == id
    former_shape = data[indicees, start_id:features].shape
    #delete index 0 and 3 or 9, containing the label and the user id
    client_x = np.delete( data[indicees, start_id:features], label_id-1, 1 ).reshape(former_shape[0], former_shape[1]-1)
    #scale 
    client_x = scaler.transform(client_x)
    client_y = data[indicees, label_id].reshape(-1, 1)
    
    if len(client_x) > 1:
        X_train, X_test, y_train, y_test = train_test_split(client_x, client_y, test_size=0.2, random_state=42)    
        ds_train = create_dataset(X_train, y_train, use_tff)
        ds_test = create_dataset(X_test, y_test, use_tff)
        print("Client {}: Created  dataset".format(id))

        train_data.append(ds_train)
        test_data.append(ds_test)
    else:
        print("Could not generate datasets for client {} as there is just one entry in X_train".format(id))
        client_ids.remove(id)

In [ ]:
# Check format for TFF: needs to be in shape(None, dim)
#like eg:
# (TensorSpec(shape=(None, 3), dtype=tf.float64, name=None),
#  TensorSpec(shape=(None, 1), dtype=tf.float64, name=None)

#Check format for unfederated Learning: shape(batchsize, dim)
print(train_data[0].element_spec)
print(test_data[0].element_spec)

In [ ]:
def get_not_bn_idx(trainable_variables):
  new_trainable_weights = []
  train_var_idx = []
  for idx, bn_weights in enumerate(trainable_variables):
    if 'batch_normalization' not in bn_weights.name:
      train_var_idx.append(idx)
  return train_var_idx

def get_bn_idx(trainable_variables):
  new_trainable_weights = []
  train_var_idx = []
  for idx, bn_weights in enumerate(trainable_variables):
    if 'batch_normalization' in bn_weights.name:
      train_var_idx.append(idx)
  return train_var_idx

def get_weights_by_idx(trainable_variables, var_ids):
  new_weights = []  
  for i in var_ids:
    new_weights.append(trainable_variables[i])
  return new_weights

def create_keras_model(input_dim):
    return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(input_dim,)),
      tf.keras.layers.Dense(500, activation=tf.nn.relu),
      tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'),
    ])

def create_keras_bn_model(input_dim):
    return tf.keras.models.Sequential([
      tf.keras.layers.BatchNormalization(input_shape=(input_dim,)),
      tf.keras.layers.Dense(500, activation=tf.nn.relu),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'),
    ])

# Each time the next method is called, the server model is broadcast to each client using a broadcast function. 
# For each client, one epoch of local training is performed via the tf.keras.optimizers.Optimizer.apply_gradients method of the client optimizer. 
# Each client computes the difference between the client model after training and the initial broadcast model. 
# These model deltas are then aggregated at the server using some aggregation function. 
# The aggregate model delta is applied at the server by using the tf.keras.optimizers.Optimizer.apply_gradients method of the server optimizer.
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
    if use_bn:
      keras_model = create_keras_bn_model(test_data[0].element_spec[0].shape[1])
    else:
      keras_model = create_keras_model(test_data[0].element_spec[0].shape[1])
    return tff.learning.from_keras_model(
      keras_model,
      input_spec = train_data[0].element_spec,
      loss = tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics = [tf.keras.metrics.SparseCategoricalAccuracy(), 
               tf.keras.metrics.SparseTopKCategoricalAccuracy(k=k)]) 
    

def map_weights_ids():
  model = create_keras_bn_model(train_data[0].element_spec[0].shape[1])
  trainable_variables = model.trainable_variables
  all_weights_name = []
  train_weights_ids = []

  for layer in model.layers:
    for weight in layer.weights:
      all_weights_name.append(weight.name)

  for var in trainable_variables:
    for idx, name in enumerate(all_weights_name):
      if var.name == name:
        if 'batch_normalization' not in name:
          train_weights_ids.append(idx)

  return train_weights_ids

In [ ]:
def run_federated():

  # ---custom fed avg implementation for batch normalization-----
  # --------------------------START------------------------------
  @tff.tf_computation
  def server_init():
    model = model_fn()
    trainable_variables = model.trainable_variables
    non_bn_ids = get_not_bn_idx(trainable_variables)
    non_bn_weights = get_weights_by_idx(trainable_variables, non_bn_ids)
    return non_bn_weights

  @tf.function
  def server_update(model, mean_client_weights):
    """Updates the server model weights as the average of the client model weights."""
    updated_model_weights = []
    trainable_variables = model.trainable_variables
    
    non_bn_ids = get_not_bn_idx(trainable_variables)
    bn_ids = get_bn_idx(trainable_variables)
    non_bn_weights = get_weights_by_idx(trainable_variables, non_bn_ids)
    bn_weights = get_weights_by_idx(trainable_variables, bn_ids)
    
    tf.nest.map_structure(lambda x, y: x.assign(y),
                          non_bn_weights, mean_client_weights)
    for i in range(len(trainable_variables)):
      if i in non_bn_ids:
        j = non_bn_ids.index(i)
        updated_model_weights.append(non_bn_weights[j])
      else: 
        j = bn_ids.index(i)
        updated_model_weights.append(bn_weights[j])
    return non_bn_weights


  @tf.function
  def client_update(model, dataset, server_weights, client_optimizer):
    updated_clients_weights = []
    trainable_variables = model.trainable_variables
    non_bn_ids = get_not_bn_idx(trainable_variables)
    bn_ids = get_bn_idx(trainable_variables)
    non_bn_weights = get_weights_by_idx(trainable_variables, non_bn_ids)
    bn_weights = get_weights_by_idx(trainable_variables, bn_ids)

    # Assign the mean client weights to the server model.
    tf.nest.map_structure(lambda x, y: x.assign(y),
                          non_bn_weights, server_weights)
    
    for i in range(len(trainable_variables)):
      if i in non_bn_ids:
        j = non_bn_ids.index(i)
        updated_clients_weights.append(non_bn_weights[j])
      else: 
        j = bn_ids.index(i)
        updated_clients_weights.append(bn_weights[j])
    
    client_weights = updated_clients_weights

    for epoch in range(1):
      for batch in dataset:
        with tf.GradientTape() as tape:
          outputs = model.forward_pass(batch)
        grads = tape.gradient(outputs.loss, client_weights)
        grads_and_vars = zip(grads, client_weights)
        client_optimizer.apply_gradients(grads_and_vars)

    return non_bn_weights

  whimsy_model = model_fn()
  tf_dataset_type = tff.SequenceType(whimsy_model.input_spec)
  model_weights_type = server_init.type_signature.result
  federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
  federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)
  print(federated_server_type)
  print(federated_dataset_type)

  @tff.tf_computation(tf_dataset_type, model_weights_type)
  def client_update_fn(tf_dataset, server_weights):
    model = model_fn()
    client_optimizer = tf.keras.optimizers.SGD(learning_rate= learning_rate, momentum=momentum, nesterov=nesterov)
    return client_update(model, tf_dataset, server_weights, client_optimizer)

  @tff.tf_computation(model_weights_type)
  def server_update_fn(mean_client_weights):
    model = model_fn()
    return server_update(model, mean_client_weights)

  @tff.federated_computation
  def initialize_fn():
    return tff.federated_value(server_init(), tff.SERVER)


  @tff.federated_computation(federated_server_type, federated_dataset_type)
  def next_fn(server_weights, federated_dataset):

    # Broadcast the server weights to the clients.
    server_weights_at_client = tff.federated_broadcast(server_weights)

    # Each client computes their updated weights.
    client_weights = tff.federated_map(
        client_update_fn, (federated_dataset, server_weights_at_client))
    
    # The server averages these updates.
    mean_client_weights = tff.federated_mean(client_weights)

    # # The server updates its model.
    server_weights = tff.federated_map(server_update_fn, mean_client_weights)
    return server_weights

  def evaluate(server_state, train_data, test_data, trainable_ids):
    server_weights = []
    acc_mean, loss_mean, k_acc_mean = [], [], []
    model = create_keras_bn_model(test_data[0].element_spec[0].shape[1])
    model.compile(
          loss = tf.keras.losses.SparseCategoricalCrossentropy(),
          metrics = [tf.keras.metrics.SparseCategoricalAccuracy(), 
                  tf.keras.metrics.SparseTopKCategoricalAccuracy(k=k)]  
    )
    weights = model.get_weights()

    for idx, weight_id in enumerate(trainable_ids):
      weights[weight_id] = np.array(server_state[idx])
      
    model.set_weights(weights)
    print("\t--Training--\t")
    for batch in train_data:
      loss, acc, k_acc = model.evaluate(batch, batch_size=BATCH_SIZE, verbose=1)
      loss_mean.append(loss)
      acc_mean.append(acc)
      k_acc_mean.append(k_acc)
    train_loss = statistics.mean(loss_mean)
    train_acc = statistics.mean(acc_mean)
    train_k_acc = statistics.mean(k_acc_mean)
    acc_mean, loss_mean, k_acc_mean = [], [], []

    print("\t--Testing--\t")
    for batch in test_data:
      loss, acc, top_k_acc = model.evaluate(batch, batch_size=BATCH_SIZE, verbose=1)
      loss_mean.append(loss)
      acc_mean.append(acc)
      k_acc_mean.append(k_acc)
    test_loss = statistics.mean(loss_mean)
    test_acc = statistics.mean(acc_mean)
    test_k_acc = statistics.mean(k_acc_mean)
    return {"Train_acc":train_acc, f"Train_{k}_acc":train_k_acc, "Train_loss":train_loss, "Test_acc":test_acc, f"Test_{k}_acc":test_k_acc,  "Test_loss":test_loss,}
    # -------------------END---------------------

  with tf.device('/gpu:0'):
    if use_bn:    #if batch normalization dont update the clients weights of the batch normalization layer. 
        #returns just server state-> 
        trainable_ids = map_weights_ids()
        fed_avg = tff.templates.IterativeProcess(
                              initialize_fn=initialize_fn,
                              next_fn=next_fn
                              )
        
        state = fed_avg.initialize()
        with summary_writer.as_default():
            for round_num in range(EPOCHS):  
                state = fed_avg.next(state, train_data)
                metrics = evaluate(state, train_data, test_data, trainable_ids)
                for name, value in metrics.items():
                  tf.summary.scalar(name, value, step=round_num)
                  print(round_num, name, value)
              
    else:
      fed_avg = tff.learning.build_federated_averaging_process(
          model_fn,
          client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate= learning_rate, momentum=momentum, nesterov=nesterov), 
          server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate= 1.0)
          )

      state = fed_avg.initialize()
      with summary_writer.as_default():
          for round_num in range(EPOCHS):
              state, metrics = fed_avg.next(state, train_data)
              # Note: training metrics reported by the iterative training process 
              #generally reflect the performance of the model at the beginning of the training round
              for name, value in metrics['train'].items():
                  tf.summary.scalar(name, value, step=round_num)
                  print(round_num, name, value)
                    
              evaluation = tff.learning.build_federated_evaluation(model_fn)  
              test_metrics = evaluation(state.model, test_data)
              for name, value in test_metrics.items():
                  tf.summary.scalar(name, value, step=round_num)
                  print(round_num, name, value)


In [ ]:
summary_writer = tf.summary.create_file_writer(tf_log_dir + "/" + file + "-" + str(CLIENTS) + "/federated-" + datetime.now().strftime("%Y%m%d-%H%M%S"))
run_federated()

## Unfederated Trainings

In [ ]:
def run_unfederated(ds_train, ds_test, ds_val, input_dim):

    early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                min_delta=0.01, 
                                patience=2, 
                                verbose=1, 
                                mode='auto', 
                                baseline=None, 
                                restore_best_weights=True)
    if use_bn:
      model = BNModel(NUM_CLASSES)
    else:
      model = FLModel(NUM_CLASSES)
    model.compile(
                optimizer= client_optimizer, 
                loss= "sparse_categorical_crossentropy", 
                metrics= [
                          sparseCategoricalAcc, 
                          sparseTopKCategoricalAccuracy
                          ]
                )
    for epoch in range(EPOCHS):
        with tf.device('/gpu:0'):
            history = model.fit(
                              ds_train,
                              steps_per_epoch=64, 
                              validation_data = ds_val, 
                              verbose=0,
                              callbacks = [early_stopping_callback])

        loss = round(history.history["loss"][0], 8)
        acc = round(history.history["sparse_categorical_accuracy"][0], 8)
        k_acc =  round(history.history["sparse_top_k_categorical_accuracy"][0], 8)
        val_loss =  round(history.history['val_loss'][0], 8)
        val_acc = round(history.history['val_sparse_categorical_accuracy'][0], 8)
        val_k_acc =  round(history.history['val_sparse_top_k_categorical_accuracy'][0], 8)

        with tf.device('/gpu:0'):
            test_loss, test_acc, test_k_acc = model.evaluate(ds_test, batch_size=BATCH_SIZE, verbose=0)
        
        test_loss = round(test_loss, 8)
        test_acc = round(test_acc, 8)
        test_k_acc = round(test_k_acc, 8)

        with summary_writer.as_default():
            tf.summary.scalar("Loss/train", loss, step=epoch)
            tf.summary.scalar("Acc/train", acc, step=epoch)
            tf.summary.scalar("K_acc/train", k_acc, step=epoch)

            tf.summary.scalar("Loss/validation", val_loss, step=epoch)
            tf.summary.scalar("Acc/validation", val_acc, step=epoch)
            tf.summary.scalar("K_acc/validation", val_k_acc, step=epoch)

            tf.summary.scalar("Loss/test", test_loss, step=epoch)
            tf.summary.scalar("Acc/test", test_acc, step=epoch)
            tf.summary.scalar("K_acc/test", test_k_acc, step=epoch)

        print(
          f'Epoch: {epoch},\n'
          f'Train Loss:\t{loss}, '
          f'Train Accuracy:\t{acc}, '
          f'Train Top 5 Accuracy:\t{k_acc}\n'
          f'Validation Loss:\t{val_loss}, '
          f'Validation Accuracy:\t{val_acc}, '
          f'Validation Top 5 Accuracy:\t{val_k_acc}\n'
          f'Test Loss:\t{test_loss}, '
          f'Test Accuracy:\t{test_acc} '
          f'Test Top 5 Accuracy:\t{test_k_acc}'
          f'\n--------------------------------------------------------------------------------------------------------------------------\n'
        )

In [ ]:
summary_writer = tf.summary.create_file_writer(tf_log_dir + "/" + file + "-" + str(CLIENTS) + "/unfederated-" + datetime.now().strftime("%Y%m%d-%H%M%S"))
#get same client ids, as with tff
mask = np.isin(data[:, 0], client_ids)
x = data[mask].copy() 

rununfederated_train, unfederated_test, unfederated_val = create_unfederated_dataset(x, reader.get_features())
_unfederated(unfederated_train, unfederated_test, unfederated_val,  (reader.get_features()-2))

In [ ]:
%tensorboard --logdir {tf_log_dir}